## Pre_processing code


Note: we are still working on this part, so this is not the final version.

---



---



In [ ]:
# Suggested Packages
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import re

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Loading the data set - training data.
from sklearn.datasets import fetch_20newsgroups

# mydata_train = fetch_20newsgroups(subset='train', shuffle=True)
mydata_train = fetch_20newsgroups(subset='train', shuffle=True, remove = ('headers', 'footers', 'quotes'))

In [ ]:
mydata_train_df = pd.DataFrame({'data': mydata_train.data, 'target': mydata_train.target})

mydata_train_df.data[1]

"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

In [ ]:
mydata_train_df.head()

,data,target
0,I was wondering if anyone out there could enli...,7
1,A fair number of brave souls who upgraded thei...,4
2,"well folks, my mac plus finally gave up the gh...",4
3,\nDo you have Weitek's address/phone number? ...,1
4,"From article <C5owCB.n3p@world.std.com>, by to...",14


In [ ]:
# Printing all the categories
mydata_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
# Lowercasing
#mydata_train_df['data']=mydata_train_df['data'].str.lower()
#mydata_train_df.head()

In [ ]:
# Remove URL

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)


mydata_train_df['data'] = mydata_train_df['data'].apply(remove_urls)
mydata_train_df.head()

,data,target
0,I was wondering if anyone out there could enli...,7
1,A fair number of brave souls who upgraded thei...,4
2,"well folks, my mac plus finally gave up the gh...",4
3,\nDo you have Weitek's address/phone number? ...,1
4,"From article <C5owCB.n3p@world.std.com>, by to...",14


In [ ]:
# Remove email address:

def remove_email(text):
    email_pattern = re.compile(r'\S+@\S+(?:\.\S+)+')
    return email_pattern.sub(r'', text)


mydata_train_df['data'] = mydata_train_df['data'].apply(remove_email)
mydata_train_df.head()


,data,target
0,I was wondering if anyone out there could enli...,7
1,A fair number of brave souls who upgraded thei...,4
2,"well folks, my mac plus finally gave up the gh...",4
3,\nDo you have Weitek's address/phone number? ...,1
4,From article by (Tom A Baker):\n\n\nMy under...,14


In [ ]:
# Remove Extra Whitespaces

def remove_whitespace(text):
    return  " ".join(text.split())

white_space=mydata_train_df['data'].apply(remove_whitespace)
white_space.head()

0    I was wondering if anyone out there could enli...
1    A fair number of brave souls who upgraded thei...
2    well folks, my mac plus finally gave up the gh...
3    Do you have Weitek's address/phone number? I'd...
4    From article by (Tom A Baker): My understandin...
Name: data, dtype: object

In [ ]:
# Tokenization

from nltk import word_tokenize
to_token=white_space.apply(lambda X: word_tokenize(X))
to_token.head()

0    [I, was, wondering, if, anyone, out, there, co...
1    [A, fair, number, of, brave, souls, who, upgra...
2    [well, folks, ,, my, mac, plus, finally, gave,...
3    [Do, you, have, Weitek, 's, address/phone, num...
4    [From, article, by, (, Tom, A, Baker, ), :, My...
Name: data, dtype: object

In [ ]:
# Removing Stopwords

from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
english_stopwords = stopwords.words('english')

def remove_stopwords(text):
    result = []
    for token in text:
        if token not in english_stopwords:
            result.append(token)
            
    return result

word_stop = to_token.apply(remove_stopwords)
word_stop.head()

0    [I, wondering, anyone, could, enlighten, car, ...
1    [A, fair, number, brave, souls, upgraded, SI, ...
2    [well, folks, ,, mac, plus, finally, gave, gho...
3    [Do, Weitek, 's, address/phone, number, ?, I, ...
4    [From, article, (, Tom, A, Baker, ), :, My, un...
Name: data, dtype: object

In [ ]:
# Removing Punctuations

from nltk.tokenize import RegexpTokenizer

def remove_punct(text):
    
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

pun_word = word_stop.apply(remove_punct)
pun_word.head()

0    [I, wondering, anyone, could, enlighten, car, ...
1    [A, fair, number, brave, souls, upgraded, SI, ...
2    [well, folks, mac, plus, finally, gave, ghost,...
3    [Do, Weitek, s, address, phone, number, I, d, ...
4    [From, article, Tom, A, Baker, My, understandi...
Name: data, dtype: object

In [ ]:
# Lemmatization

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

def lemmatization(text):
    
    result=[]
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()
        
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
       
        result.append(wordnet.lemmatize(token,pos))
      
        
    return result


word_lem=pun_word.apply(lemmatization)
word_lem.head()

0    [I, wonder, anyone, could, enlighten, car, I, ...
1    [A, fair, number, brave, soul, upgrade, SI, cl...
2    [well, folk, mac, plus, finally, give, ghost, ...
3    [Do, Weitek, s, address, phone, number, I, d, ...
4    [From, article, Tom, A, Baker, My, understandi...
Name: data, dtype: object

In [ ]:
# Stemming

from nltk.stem import PorterStemmer

def stemming(text):
    porter = PorterStemmer()
    
    result=[]
    for word in text:
        result.append(porter.stem(word))
    return result


word_stem = word_lem.apply(stemming)
word_stem.head()

0    [I, wonder, anyon, could, enlighten, car, I, s...
1    [A, fair, number, brave, soul, upgrad, SI, clo...
2    [well, folk, mac, plu, final, give, ghost, wee...
3    [Do, weitek, s, address, phone, number, I, d, ...
4    [from, articl, tom, A, baker, My, understand, ...
Name: data, dtype: object

In [ ]:
# POS_tag
from nltk.tokenize import word_tokenize
from nltk import pos_tag
  
# convert text into word_tokens with their tags
def pos_tagging(text):
     
    result=[]
   
    for token,tag in pos_tag(text):
         
        if not tag == 'NN': result.append(token.lower())
        else: result.append(token)
       

        
    return result


word_pos = pun_word.apply(pos_tagging)
word_pos.head()

0    [i, wondering, anyone, could, enlighten, car, ...
1    [a, fair, number, brave, souls, upgraded, si, ...
2    [well, folks, mac, plus, finally, gave, ghost,...
3    [do, weitek, s, address, phone, number, i, d, ...
4    [from, article, tom, a, baker, my, understandi...
Name: data, dtype: object